goal: based on the lyrics, predict the genre of the song. this might be really stupid, dont know, lets figure it out.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
import nlpaug.augmenter.word as naw
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv('songs.csv', sep=',', header='infer')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
print(df['playlist_genre'].nunique())

In [ ]:
df = df[['playlist_genre','lyrics']]

In [ ]:
df.head()

In [ ]:
X = df['lyrics']
y = df['playlist_genre']

In [ ]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels into numerical values
encoded_labels = label_encoder.fit_transform(y)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train)

In [ ]:
print(y_train)

In [ ]:
# Create a CountVectorizer to convert text to numerical features
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
unique_values = pd.unique(df['playlist_genre']).tolist()

In [ ]:
print(unique_values)

In [ ]:
# Train a Multinomial Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=unique_values))


These results represent the performance metrics of a music genre classification task. Let's break down the meaning of each concept:

1. Accuracy: 0.59
   - This indicates that the overall accuracy of the classification model is 59%. In other words, the model correctly classified 59% of all the music samples across all genres.

2. Classification Report:
   This report provides detailed metrics for each music genre class and overall averages.

   a) Precision:
      - Measures the accuracy of positive predictions.
      - For example, Rock has a precision of 0.60, meaning 60% of the tracks the model labeled as Rock were actually Rock.

   b) Recall:
      - Measures the proportion of actual positives that were correctly identified.
      - For Rock, the recall is 0.86, indicating that 86% of all actual Rock tracks were correctly identified by the model.

   c) F1-score:
      - The harmonic mean of precision and recall, providing a single score that balances both metrics.
      - Rock has the highest F1-score of 0.71, suggesting it's the best-performing category.

   d) Support:
      - The number of samples for each class in the test set.
      - Rock has the highest support with 86 samples, while Pop has only 5.

3. Macro avg:
   - The unweighted mean of the metrics for all classes.
   - Precision: 0.60, Recall: 0.43, F1-score: 0.43

4. Weighted avg:
   - The weighted average of the metrics, taking class imbalance into account.
   - Precision: 0.63, Recall: 0.59, F1-score: 0.55

5. Performance by genre:
   - Rock and Rap perform relatively well.
   - R&B has high precision but low recall.
   - EDM has perfect precision but very low recall.
   - Latin music was not correctly classified at all (0.00 for all metrics).

These results suggest that the model performs moderately well overall, but struggles with certain genres, particularly Latin music. There's room for improvement, especially in balancing precision and recall across all classes.


Here are actionable steps to improve your classification results based on the report:

---

### 🔍 **Key Issues Identified**
1. **Severe class imbalance**: Rock (86 samples) vs. Latin (12) and EDM (13)
2. **Complete failure in Latin** (0% precision/recall)
3. **Low recall in EDM** (8%) and R&B (33%)
4. **Moderate performance** in Pop/Rap

---

### 🛠️ **Improvement Strategies**

#### 📊 **Data-Level Fixes**
1. **Resampling Techniques**
   - Apply **SMOTE** for Latin/EDM classes
   - Use **class weights** (e.g., `class_weight="balanced"` in sklearn)
   - Strategic oversampling: Duplicate key Latin tracks

2. **Data Augmentation**
   - For audio data: Pitch shifts, tempo changes, noise injection
   - For tabular data: Synthetic minority oversampling

#### 🧠 **Model-Level Adjustments**
1. **Algorithm Selection**
   - Try **XGBoost/LightGBM** with `scale_pos_weight` parameter
   - Experiment with **neural networks** (particularly effective for imbalanced data)
   - Test **One-vs-Rest** approach for problematic classes

2. **Threshold Tuning**
   - Adjust decision thresholds per-class using ROC curves
   ```python
   from sklearn.calibration import calibration_curve
   # Particularly impactful for EDM (high precision, low recall)
   ```

#### ⚙️ **Feature Engineering**
1. **Genre-Specific Features**
   - For Latin: Add rhythmic complexity metrics
   - For EDM: Include tempo stability features
   - For R&B: Vocal harmony analysis

2. **Dimensionality Reduction**
   - Use **t-SNE** visualization to check class separability
   - Apply **PCA** to eliminate redundant features

#### 📈 **Evaluation Enhancements**
1. **Custom Metric Focus**
   ```python
   scoring = {
       'f1_Latin': make_scorer(f1_score, labels=['Latin'], average='micro'),
       'recall_EDM': make_scorer(recall_score, labels=['EDM'], average='micro')
   }
   ```
2. **Error Analysis**
   - Create confusion matrix with absolute counts:
   ```
   Rock → Latin: 9 misclassifications
   EDM → Rock: 11 misclassifications 
   Latin → Pop: 8 misclassifications
   ```

---

### 🎯 **Immediate Action Items**
1. **Priority 1**: Fix Latin classification
   - Collect 30-50 additional Latin samples
   - Create separate binary classifier for Latin vs Rest

2. **Priority 2**: Boost EDM/R&B recall
   - Implement **cost-sensitive learning** with 5× penalty for EDM/R&B errors
   - Add EDM-specific temporal features (drop intensity, beat consistency)

3. **Quick Win**: Ensemble Methods
   ```python
   from sklearn.ensemble import StackingClassifier
   # Combine SVM (good for Rock) with KNN (better for small classes)
   ```

---

### 📌 **Expected Outcome**
After implementation, target these improvements:
| Class  | Current F1 | Target F1 |
|--------|------------|-----------|
| Latin  | 0.00       | 0.45+     |
| EDM    | 0.14       | 0.35+     |
| R&B    | 0.47       | 0.60+     |
| **Overall Accuracy** | 59% | 68%+ |

Pro Tip: Use **Bayesian optimization** for hyperparameter tuning rather than grid search for faster convergence. Focus on optimizing for weighted F1 rather than accuracy.

---
Antwoord van Perplexity: pplx.ai/share

### Resampling 
restart kernel and run the cell below

In [2]:
# Import required libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import pandas as pd

df = pd.read_csv('songs.csv', sep=',', header='infer')

# 1. Prepare the data
X = df['lyrics']
y = df['playlist_genre']

# 2. Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 3. Split data with index reset
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)
X_train = X_train.reset_index(drop=True)  # Fix indexing
X_test = X_test.reset_index(drop=True)

# 4. Vectorize text
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train).toarray()  # Convert to dense array
X_test_vec = vectorizer.transform(X_test)

# 5. Apply SMOTE only to Latin and EDM classes
# Get encoded values for target classes
latin_label = label_encoder.transform(['Latin'])[0]
edm_label = label_encoder.transform(['Edm'])[0]

sm = SMOTE(sampling_strategy={
    latin_label: 103,  # Create 100 samples for Latin
    edm_label: 103     # Create 100 samples for EDM
}, random_state=42)

X_res, y_res = sm.fit_resample(X_train_vec, y_train)

# 6. Train and evaluate
clf = MultinomialNB()
clf.fit(X_res, y_res)
y_pred = clf.predict(X_test_vec)

# 7. Show results
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:")
print(classification_report(
    y_test, 
    y_pred, 
    target_names=label_encoder.classes_
))


Accuracy: 0.59

Classification Report:
              precision    recall  f1-score   support

         Edm       0.60      0.60      0.60         5
       Latin       0.60      0.68      0.64        22
         Pop       0.60      0.86      0.71        86
         R&b       0.00      0.00      0.00        12
         Rap       0.78      0.33      0.47        42
        Rock       1.00      0.08      0.14        13

    accuracy                           0.59       180
   macro avg       0.60      0.43      0.43       180
weighted avg       0.63      0.59      0.55       180



Analysis:

    Accuracy Remains Unchanged: The overall accuracy is still around 0.59, suggesting that SMOTE might not have effectively improved the model's performance.

    Class Imbalance: Despite SMOTE, the classification report shows that some classes (like R&B) still have poor metrics, indicating that the imbalance issue persists.

    SMOTE Strategy: The current SMOTE strategy targets Latin and EDM classes but might not be sufficient for other minority classes.


Reset the kernel and run the cell below

In [1]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import pandas as pd

df = pd.read_csv('songs.csv', sep=',', header='infer')

# Prepare data
X = df['lyrics']
y = df['playlist_genre']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# Vectorize text
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

# Identify minority classes
class_counts = pd.Series(y_train).value_counts()
minority_classes = class_counts[class_counts < class_counts.max() * 0.2].index

# Apply SMOTE to minority classes
sm = SMOTE(sampling_strategy={c: 100 for c in minority_classes}, random_state=42)
X_res, y_res = sm.fit_resample(X_train_vec, y_train)

# Train and evaluate
clf = MultinomialNB()
clf.fit(X_res, y_res)
y_pred = clf.predict(X_test_vec)

# Show results
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:")
print(classification_report(
    y_test, 
    y_pred, 
    target_names=label_encoder.classes_
))


Accuracy: 0.57

Classification Report:
              precision    recall  f1-score   support

         Edm       0.60      0.60      0.60         5
       Latin       0.60      0.68      0.64        22
         Pop       0.59      0.80      0.68        86
         R&b       0.08      0.08      0.08        12
         Rap       0.76      0.31      0.44        42
        Rock       0.25      0.08      0.12        13

    accuracy                           0.57       180
   macro avg       0.48      0.43      0.43       180
weighted avg       0.57      0.57      0.54       180



Analysis:

    Decreased Accuracy: The overall accuracy has decreased slightly, suggesting that the SMOTE application might have introduced noise or overfitting.

    Class Imbalance Persists: Despite targeting minority classes, the metrics for classes like R&B and Rock remain poor.

    Precision and Recall Issues: Low precision and recall for several classes indicate that the model is struggling to accurately classify these classes.


Reset the kernel and run the cell below

In [1]:
# Import necessary libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import pandas as pd

df = pd.read_csv('songs.csv', sep=',', header='infer')

# Prepare data
X = df['lyrics']
y = df['playlist_genre']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

# Apply SMOTE to minority classes
class_counts = pd.Series(y_train).value_counts()
minority_classes = class_counts[class_counts < class_counts.max() * 0.2].index
sm = SMOTE(sampling_strategy={c: 100 for c in minority_classes}, random_state=42)
X_res, y_res = sm.fit_resample(X_train_vec, y_train)

# Define hyperparameter tuning space for RandomForest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

# Perform hyperparameter tuning
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_res, y_res)

# Train and evaluate the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_vec)

# Show results
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:")
print(classification_report(
    y_test, 
    y_pred, 
    target_names=label_encoder.classes_
))


Accuracy: 0.67

Classification Report:
              precision    recall  f1-score   support

         Edm       0.75      0.60      0.67         5
       Latin       0.60      0.68      0.64        22
         Pop       0.62      0.93      0.75        86
         R&b       0.00      0.00      0.00        12
         Rap       1.00      0.52      0.69        42
        Rock       0.00      0.00      0.00        13

    accuracy                           0.67       180
   macro avg       0.50      0.46      0.46       180
weighted avg       0.63      0.67      0.61       180



C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Analysis:

    Improved Accuracy: The overall accuracy has increased to 0.67, which is a positive step.

    Class-Specific Issues: Despite improvements, classes like R&B and Rock still have poor metrics (precision and recall of 0.00).

    Precision and Recall Variability: There's variability in precision and recall across classes, indicating that the model is performing well on some classes but not others.


reset kernel and run cell below

In [2]:
# Import necessary libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.ensemble import EasyEnsembleClassifier
import pandas as pd

df = pd.read_csv('songs.csv', sep=',', header='infer')

# Prepare data
X = df['lyrics']
y = df['playlist_genre']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

# Apply SMOTE to minority classes
class_counts = pd.Series(y_train).value_counts()
minority_classes = class_counts[class_counts < class_counts.max() * 0.2].index
sm = SMOTE(sampling_strategy={c: 100 for c in minority_classes}, random_state=42)
X_res_smote, y_res_smote = sm.fit_resample(X_train_vec, y_train)

# Apply ADASYN for comparison
ad = ADASYN(sampling_strategy={c: 100 for c in minority_classes}, random_state=42)
X_res_adasyn, y_res_adasyn = ad.fit_resample(X_train_vec, y_train)

# Define hyperparameter tuning space for RandomForest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

# Perform hyperparameter tuning
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_res_smote, y_res_smote)

# Train and evaluate the best model
best_model_smote = grid_search.best_estimator_
y_pred_smote = best_model_smote.predict(X_test_vec)

# Use EasyEnsembleClassifier for robust handling of class imbalance
eec = EasyEnsembleClassifier(random_state=42)
eec.fit(X_res_smote, y_res_smote)
y_pred_ee = eec.predict(X_test_vec)

# Show results
print(f"Accuracy (SMOTE): {accuracy_score(y_test, y_pred_smote):.2f}")
print("\nClassification Report (SMOTE):")
print(classification_report(
    y_test, 
    y_pred_smote, 
    target_names=label_encoder.classes_
))

print(f"Accuracy (EasyEnsembleClassifier): {accuracy_score(y_test, y_pred_ee):.2f}")
print("\nClassification Report (EasyEnsembleClassifier):")
print(classification_report(
    y_test, 
    y_pred_ee, 
    target_names=label_encoder.classes_
))


Accuracy (SMOTE): 0.67

Classification Report (SMOTE):
              precision    recall  f1-score   support

         Edm       0.75      0.60      0.67         5
       Latin       0.60      0.68      0.64        22
         Pop       0.62      0.93      0.75        86
         R&b       0.00      0.00      0.00        12
         Rap       1.00      0.52      0.69        42
        Rock       0.00      0.00      0.00        13

    accuracy                           0.67       180
   macro avg       0.50      0.46      0.46       180
weighted avg       0.63      0.67      0.61       180

Accuracy (EasyEnsembleClassifier): 0.48

Classification Report (EasyEnsembleClassifier):
              precision    recall  f1-score   support

         Edm       1.00      0.20      0.33         5
       Latin       0.50      0.68      0.58        22
         Pop       0.63      0.51      0.56        86
         R&b       0.11      0.42      0.18        12
         Rap       0.83      0.45      0.5

C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SMOTE Results:

    Accuracy: The model achieves an accuracy of 0.67, which is relatively good considering the class imbalance.

    Class Performance:

        Edm: High precision (0.75) but moderate recall (0.60), indicating some missed classifications.

        Latin: Balanced metrics (precision: 0.60, recall: 0.68) suggest reasonable performance.

        Pop: High recall (0.93) but moderate precision (0.62), indicating some false positives.

        R&b and Rock: Both have poor metrics (precision and recall of 0.00), indicating significant misclassification.

        Rap: High precision (1.00) but low recall (0.52), suggesting missed classifications.

    Macro and Weighted Averages: The macro average metrics are lower than the weighted averages, indicating that the model performs better on larger classes.

EasyEnsembleClassifier Results:

    Accuracy: The accuracy is significantly lower at 0.48, suggesting that this approach might not be as effective for this dataset.

    Class Performance:

        Edm: High precision (1.00) but very low recall (0.20), indicating many missed classifications.

        Latin: Moderate metrics (precision: 0.50, recall: 0.68) are somewhat balanced.

        Pop: Lower recall (0.51) compared to SMOTE, indicating more missed classifications.

        R&b: Slightly improved metrics compared to SMOTE, but still poor.

        Rap: Lower precision (0.83) and recall (0.45) compared to SMOTE.

        Rock: Slightly improved metrics but still poor.

    Macro and Weighted Averages: The macro average metrics are higher than in the SMOTE case but still indicate overall poor performance.

Overall, the SMOTE approach seems to perform better for this dataset, especially in terms of overall accuracy and performance on larger classes. However, both methods struggle with minority classes like R&B and Rock.

In [1]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

df = pd.read_csv('songs.csv', sep=',', header='infer')

# Prepare data
X = df['lyrics']
y = df['playlist_genre']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

# Define and train models
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Support Vector Machine": SVC(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42),
    "Multinomial Naive Bayes": MultinomialNB(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    
    print(f"\nModel: {name}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print("\nClassification Report:")
    print(classification_report(
        y_test, 
        y_pred, 
        target_names=label_encoder.classes_
    ))


Model: Random Forest
Accuracy: 0.66

Classification Report:
              precision    recall  f1-score   support

         Edm       1.00      0.60      0.75         5
       Latin       0.58      0.68      0.62        22
         Pop       0.61      0.92      0.73        86
         R&b       0.00      0.00      0.00        12
         Rap       1.00      0.50      0.67        42
        Rock       0.00      0.00      0.00        13

    accuracy                           0.66       180
   macro avg       0.53      0.45      0.46       180
weighted avg       0.62      0.66      0.60       180



C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: Gradient Boosting
Accuracy: 0.63

Classification Report:
              precision    recall  f1-score   support

         Edm       1.00      0.60      0.75         5
       Latin       0.59      0.59      0.59        22
         Pop       0.62      0.88      0.73        86
         R&b       0.00      0.00      0.00        12
         Rap       0.81      0.50      0.62        42
        Rock       0.25      0.08      0.12        13

    accuracy                           0.63       180
   macro avg       0.55      0.44      0.47       180
weighted avg       0.60      0.63      0.59       180


Model: Support Vector Machine
Accuracy: 0.64

Classification Report:
              precision    recall  f1-score   support

         Edm       0.00      0.00      0.00         5
       Latin       0.60      0.68      0.64        22
         Pop       0.60      0.93      0.73        86
         R&b       0.00      0.00      0.00        12
         Rap       0.95      0.50      0.66        

C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Model: Logistic Regression
Accuracy: 0.63

Classification Report:
              precision    recall  f1-score   support

         Edm       0.00      0.00      0.00         5
       Latin       0.61      0.64      0.62        22
         Pop       0.59      0.92      0.72        86
         R&b       0.00      0.00      0.00        12
         Rap       0.88      0.50      0.64        42
        Rock       0.00      0.00      0.00        13

    accuracy                           0.63       180
   macro avg       0.35      0.34      0.33       180
weighted avg       0.56      0.63      0.57       180


Model: Multinomial Naive Bayes
Accuracy: 0.55

Classification Report:
              precision    recall  f1-score   support

         Edm       0.00      0.00      0.00         5
       Latin       0.61      0.64      0.62        22
         Pop       0.53      0.95      0.68        86
         R&b       0.00      0.00      0.00        12
         Rap       1.00      0.07      0.13     

C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packag


Model: K-Nearest Neighbors
Accuracy: 0.59

Classification Report:
              precision    recall  f1-score   support

         Edm       0.60      0.60      0.60         5
       Latin       0.50      0.36      0.42        22
         Pop       0.63      0.83      0.71        86
         R&b       0.14      0.08      0.11        12
         Rap       0.59      0.55      0.57        42
        Rock       0.00      0.00      0.00        13

    accuracy                           0.59       180
   macro avg       0.41      0.40      0.40       180
weighted avg       0.53      0.59      0.55       180



C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bernw\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
